<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Part 5: Load Historica data to emulate a production system

### Contents

- [1.0 Install Python Packages](#setup)
- [2.0 Configure Credentials](#credentials)
- [3.0 OpenScale configuration](#openscale)
- [4.0 Get Subscriptions](#subscription)
- [5.0 Historical Data](#historical)

# 1.0 Install Python Packages

In [2]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install --upgrade ibm-ai-openscale==2.2.1 --no-cache --user | tail -n 1
!pip install --upgrade watson-machine-learning-client-V4==1.0.95 | tail -n 1
!pip install --upgrade pyspark==2.3 | tail -n 1

### Action: restart the kernel!

In [1]:
import warnings
warnings.filterwarnings('ignore')

# 2.0 Configure credentials <a name="credentials"></a>

<font color=red>Replace the `username` and `password` values of `************` with your Cloud Pak for Data `username` and `password`. The value for `url` should match the `url` for your Cloud Pak for Data cluster, which you can get from the browser address bar (be sure to include the 'https://'.</font> The credentials should look something like this (these are example values, not the ones you will use):

```
WOS_CREDENTIALS = {
                   "url": "https://zen.clusterid.us-south.containers.appdomain.cloud",
                   "username": "cp4duser",
                   "password" : "cp4dpass"
                  }

```

**NOTE: Make sure that there is no trailing forward slash / in the url**

In [2]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.omid-v16-2bef1f4b4097001da9502000c44fc2b2-0000.us-south.containers.appdomain.cloud",
    "username": "jrtorres",
    "password": "*******"
}

In [3]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='3.0.0'

Lets retrieve the variables for the model and deployment we set up in the initial setup notebook. **If the output does not show any values, check to ensure you have completed the initial setup before continuing.**

In [5]:
%store -r MODEL_NAME
%store -r DEPLOYMENT_NAME
%store -r DEFAULT_SPACE
%store -r model_uid
%store -r binding_uid

print("Model Name: ", MODEL_NAME, ". Deployment Name: ", DEPLOYMENT_NAME, ". Deployment Space: ", DEFAULT_SPACE)
print("Model UID: ", model_uid, ". Binding ID: ", binding_uid)

Model Name:  JRTCreditRiskRFModel-1 . Deployment Name:  RFCreditRiskModelOnlineDep . Deployment Space:  c1b077ce-c625-4861-85e2-6a7273620589
Model UID:  e31b0f84-72c7-47aa-9872-64d04dc70ddf . Binding ID:  999


# 3.0 Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and configure OpenScale

In [6]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [7]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [8]:
ai_client = APIClient4ICP(WOS_CREDENTIALS)
ai_client.version

'2.1.21'

# 4.0 Get Subscription <a name="subscription"></a>

In [9]:
subscription = None

if subscription is None:
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            print("Found existing subscription.")
            subscription = ai_client.data_mart.subscriptions.get(sub)
            subscription_uid = sub
            print("subscription_uid: " + sub)
if subscription is None:
    print("No subscription found. Please run openscale-initial-setup.ipynb to configure.")

Found existing subscription.
subscription_uid: aa6d705b-844b-4eef-a7fd-7cd8fad182be


### Set Deployment UID

In [10]:
wml_client.set.default_space(DEFAULT_SPACE)

'SUCCESS'

In [11]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    print(deployment['entity']['name'])
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break
        
print(deployment_uid)

WOS-INTERNAL-2d725963-a8af-4c41-821b-15619ae53416
RFCreditRiskModelOnlineDep
0e003bf2-9013-424b-b1a7-8b77ff555427


In [12]:
binding_uid = None

binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
if binding_uid is None:
    print("Adding binding:")
    binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance4ICP(wml_credentials=WML_CREDENTIALS))
    bindings_details = ai_client.data_mart.bindings.get_details()
else:
    print("Found binding:")
binding_uid

Found binding:


'999'

In [13]:
wml_models = wml_client.repository.get_model_details()
model_uid = None

for model_in in wml_models['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break
        
print(model_uid)

e31b0f84-72c7-47aa-9872-64d04dc70ddf


# 5.0 Historical data <a name="historical"></a>

 ## 5.1 Insert historical payloads

The next section of the notebook downloads and writes historical data to the payload and measurement tables to simulate a production model that has been monitored and receiving regular traffic for the last seven days. This historical data can be viewed in the Watson OpenScale user interface. The code uses the Python and REST APIs to write this data.

In [14]:
!rm history_payloads_with_transaction_*.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_0.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_1.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_2.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_3.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_4.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_5.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_6.json


--2020-11-16 22:59:37--  https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_0.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5209237 (5.0M) [text/plain]
Saving to: ‘history_payloads_with_transaction_id_0.json’

100%[======================================>] 5,209,237   --.-K/s   in 0.07s   

2020-11-16 22:59:37 (74.4 MB/s) - ‘history_payloads_with_transaction_id_0.json’ saved [5209237/5209237]

--2020-11-16 22:59:38--  https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_payloads_with_transaction_id_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
H

In [21]:
historyDays = 7

In [15]:
from ibm_ai_openscale.utils.inject_demo_data import DemoData
import os
import datetime
import time

historicalData = DemoData(WOS_CREDENTIALS, ai_client)
historical_data_path=os.getcwd()

historicalData.load_historical_scoring_payload(subscription, deployment_uid,file_path=historical_data_path, day_template="history_payloads_with_transaction_id_{}.json" )

Loading historical scoring payload...
Day 0 injection.
Daily loading finished.
Day 1 injection.
Daily loading finished.
Day 2 injection.
Daily loading finished.
Day 3 injection.
Daily loading finished.
Day 4 injection.
Daily loading finished.
Day 5 injection.
Daily loading finished.
Day 6 injection.
Daily loading finished.


In [16]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

00000000-0000-0000-0000-000000000000


In [17]:
performance_metrics_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

https://zen-cpd-zen.omid-v16-2bef1f4b4097001da9502000c44fc2b2-0000.us-south.containers.appdomain.cloud/v1/data_marts/00000000-0000-0000-0000-000000000000/metrics


## 5.2 Insert historical fairness metrics

In [18]:
!rm history_fairness.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_fairness.json

--2020-11-16 23:01:34--  https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_fairness.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1424078 (1.4M) [text/plain]
Saving to: ‘history_fairness.json’

100%[======================================>] 1,424,078   --.-K/s   in 0.02s   

2020-11-16 23:01:35 (60.8 MB/s) - ‘history_fairness.json’ saved [1424078/1424078]



In [23]:
import requests
import datetime
import time
from requests.auth import HTTPBasicAuth

def create_token():
    header = {
                    "Content-Type": "application/x-www-form-urlencoded",
                    "Accept": "application/json"
    }

    response = requests.Session().get(
            WOS_CREDENTIALS['url'] + '/v1/preauth/validateAuth',
            headers=header,
            auth=HTTPBasicAuth(
                WOS_CREDENTIALS['username'],
                WOS_CREDENTIALS['password']
            ),
            verify=False)

    response = handle_response(200, 'access token', response, True)
    token = response['accessToken']

    return token

In [24]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('history_fairness.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed
        
        qualityMetric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': payloads[index]
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers, verify=False)

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## 5.3 Insert historical debias metrics

In [25]:
!rm history_debias.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_debias.json

--2020-11-16 23:03:45--  https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_debias.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 565971 (553K) [text/plain]
Saving to: ‘history_debias.json’

100%[======================================>] 565,971     --.-K/s   in 0.01s   

2020-11-16 23:03:46 (49.0 MB/s) - ‘history_debias.json’ saved [565971/565971]



In [26]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('history_debias.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed

        qualityMetric = {
            'metric_type': 'debiased_fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': payloads[index]
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers, verify=False)
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## 5.4 Insert historical quality metrics

In [27]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.7,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.7
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers, verify=False)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## 5.5 Insert historical confusion matrixes

In [28]:
!rm history_quality_metrics.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_quality_metrics.json

--2020-11-16 23:04:29--  https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_quality_metrics.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80099 (78K) [text/plain]
Saving to: ‘history_quality_metrics.json’

100%[======================================>] 80,099      --.-K/s   in 0.002s  

2020-11-16 23:04:29 (38.9 MB/s) - ‘history_quality_metrics.json’ saved [80099/80099]



In [29]:
measurements_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/measurements'
print(measurements_url)

https://zen-cpd-zen.omid-v16-2bef1f4b4097001da9502000c44fc2b2-0000.us-south.containers.appdomain.cloud/v1/data_marts/00000000-0000-0000-0000-000000000000/measurements


In [30]:
with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)

for day in range(historyDays):
    index = 0
    measurments = []
    print('Day', day + 1)
    
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')

        measurement = {
            "monitor_definition_id": 'quality',
            "binding_id": subscription.binding_uid,
            "subscription_id": subscription.uid,
            "asset_id": subscription.source_uid,
            'metrics': [records[index]['metrics']],
            'sources': [records[index]['sources']],
            'timestamp': score_time
        }

        measurments.append(measurement)
        index+=1

    response = requests.post(measurements_url, json=measurments, headers=ai_client._get_headers(), verify=False)

print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## 5.6 Insert historical performance metrics

In [31]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(60, 600)
        score_resp = random.uniform(60, 300)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers, verify=False)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## 5.7 Insert historical manual labeling

In [32]:
manual_labeling_url = WOS_CREDENTIALS['url'] + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/manual_labelings'
print(manual_labeling_url)

https://zen-cpd-zen.omid-v16-2bef1f4b4097001da9502000c44fc2b2-0000.us-south.containers.appdomain.cloud/v1/data_marts/00000000-0000-0000-0000-000000000000/manual_labelings


In [33]:
!rm history_manual_labeling.json
!wget https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_manual_labeling.json

--2020-11-16 23:05:20--  https://raw.githubusercontent.com/IBM/credit-risk-workshop-cpd/master/data/openscale/history_manual_labeling.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 947956 (926K) [text/plain]
Saving to: ‘history_manual_labeling.json’

100%[======================================>] 947,956     --.-K/s   in 0.01s   

2020-11-16 23:05:21 (74.7 MB/s) - ‘history_manual_labeling.json’ saved [947956/947956]



In [34]:
iam_token = create_token()
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('history_manual_labeling.json', 'r') as history_file:
    records = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    record_json = []
    for hour in range(24):
        for record in records:
            if record['fastpath_history_day'] == day and record['fastpath_history_hour'] == hour:
                record['binding_id'] = binding_uid
                record['subscription_id'] = model_uid
                record['asset_revision'] = model_uid
                record['deployment_id'] = deployment_uid
                record['scoring_timestamp'] = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
                record_json.append(record)
    response = requests.post(manual_labeling_url, json=record_json, headers=iam_headers, verify=False)

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## 5.8 Additional data to help debugging

In [35]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)

Datamart: 00000000-0000-0000-0000-000000000000
Model: e31b0f84-72c7-47aa-9872-64d04dc70ddf
Deployment: 0e003bf2-9013-424b-b1a7-8b77ff555427
Binding: 999


## 5.9 Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [36]:
payload_data = subscription.payload_logging.get_table_content(limit=10)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability'])

,scoring_id,predictedLabel,probability
0,a00b611b-98dc-457e-858d-50463bc3649a-1,No Risk,"[0.8286570679154849, 0.1713429320845151]"
1,a00b611b-98dc-457e-858d-50463bc3649a-2,No Risk,"[0.7240212927744534, 0.2759787072255466]"
2,a00b611b-98dc-457e-858d-50463bc3649a-3,Risk,"[0.4827643972418857, 0.5172356027581143]"
3,a00b611b-98dc-457e-858d-50463bc3649a-4,No Risk,"[0.8496941666556259, 0.15030583334437406]"
4,a00b611b-98dc-457e-858d-50463bc3649a-5,No Risk,"[0.6798974188806356, 0.3201025811193644]"
5,a00b611b-98dc-457e-858d-50463bc3649a-6,Risk,"[0.13586103093852175, 0.8641389690614781]"
6,a00b611b-98dc-457e-858d-50463bc3649a-7,Risk,"[0.3855837076852059, 0.614416292314794]"
7,a00b611b-98dc-457e-858d-50463bc3649a-8,No Risk,"[0.7156795163046346, 0.2843204836953655]"
8,a00b611b-98dc-457e-858d-50463bc3649a-9,No Risk,"[0.8734498086143722, 0.12655019138562779]"
9,a00b611b-98dc-457e-858d-50463bc3649a-10,No Risk,"[0.6039923602717908, 0.39600763972820924]"


## Congratulations!

You have finished this section of the hands-on lab for IBM Watson OpenScale. You can now view the OpenScale dashboard by going to the Cloud Pak for Data `Home` page, and clicking `Services`. Choose the `OpenScale` tile and click the menu to `Open`. Click on the tile for the model you've created to see the monitors.

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary.


This notebook has been adapted from notebooks available at https://github.com/pmservice/ai-openscale-tutorials. 